User Settings:

In [ ]:
from glob import glob #file regexes

files = glob("*.fep")
equil = 10000
temperature = 300

Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm #for progress bars
import re #regex
from natsort import natsorted #for sorting "naturally" instead of alphabetically
import Analysis_Scripts.py

Read files

In [ ]:
files = natsorted(files)
data = readFiles(files)

In [ ]:
#data= pd.read_pickle("all_data.pkl")

Trim the data to remove pre-equilibration samples

In [ ]:
dG_f, dG_b = get_dG_fromData(data)

In [ ]:
plt.plot(dG_f.window, dG_f.dG, label='dG_fwd')
plt.plot(dG_b.window, -dG_b.dG, label='- dG_bwd')

plt.title('Fwd and bwd free energy difference by lambda (exponential estimator)')
plt.xlabel('Lambda')
plt.ylabel('delta-G')
plt.legend()
plt.savefig("discrepancy_by_lambda.png", dpi=1200)

In [ ]:
plt.vlines(dG_f.window, np.zeros(len(dG_f.window)), dG_f.dG + np.array(dG_b.dG), label="fwd - bwd", linewidth=3)

plt.legend()
plt.title('Fwd-bwd discrepencies by lambda')
plt.xlabel('Lambda')
plt.ylabel('Diff. in delta-G')
plt.savefig("hysteresis_Exp_Nov30.png", dpi=1200)

In [ ]:
print(f'The rough estimate for total dG (forward windows only) is: {np.sum(dG_f.dG).round(2)}kT. The backward estimate is {-np.sum(dG_b.dG).round(2)}kT')

Plot dE distributions for EACH complete window (may take several minutes)

In [ ]:
#split into forward and backward values for each window
backward = data.dE[~(data.up) * data.step>=equil].sort_index()*(-1)
forward = data.dE[data.up * data.step>=equil].sort_index()
print(f'equilibration Time: {equil}\n backward: {backward.mean()}, forward: {forward.mean()}') 

In [ ]:
import seaborn as sns

In [ ]:
completeWindows = np.sort(list(set(backward.index) & set(forward.index))) #those windows which have both forward and backward data

In [ ]:
for i in completeWindows:
    
    # creating a figure composed of two matplotlib.Axes objects (ax_box and ax_hist)
    f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw={"height_ratios": (.15, .85)})

    # assigning a graph to each ax
    tempDat = [forward.loc[i], backward.loc[i]]
    
    ax_box.boxplot(tempDat, vert=False)
    ax_box.set_yticklabels(["forward", "backward"])
    plt.title(f'[{np.round(i-0.004,3)} {np.round(i+0.004, 3)}]')
    # Remove x axis name for the boxplot
    ax_box.set(xlabel='')

    sns.histplot(backward.loc[i], bins=50, label="backward", ax=ax_hist);
    sns.histplot(forward.loc[i], bins=50, label="forward", ax=ax_hist, color="orange");
    
    plt.legend()
    plt.show()
    #plt.savefig(f'./diagnosticPlots/dE_SmallerWindows{np.round(i,3)}.svg')
    plt.clf()
    plt.close()